In [1]:
# Run these following 3 installs to if you don't have shapely, geopands, etc
# if you are using sagemaker, you should use the "conda_python3" kernel
#!pip install laspy
#!pip install geopandas
#!pip install shapely

In [2]:
# Run these following 2 installs to install/upgrade database interface modules
#!pip install sqlalchemy psycopg2
#!pip install psycopg2-binary
!pip install geoalchemy2

You are using pip version 10.0.1, however version 20.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
### install/upgrade folium mapping tools
# !pip install folium

In [4]:
import boto3
import json

In [5]:
import numpy as np
import pandas as pd
import psycopg2
import sqlalchemy as db
import matplotlib as plt

%matplotlib inline

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [7]:
#Need to get this instance's public address
!curl 'https://api.ipify.org?format=json'

{"ip":"34.230.156.67"}

In [8]:
#Get information from Secrets Manager, especially about the database
secretsmanager_client = boto3.client('secretsmanager')
dbSecretValues = secretsmanager_client.get_secret_value(SecretId='UrbanInstituteDevRDSParameter')

In [9]:
dbSecretValuesJson = json.loads(dbSecretValues['SecretString'])
#uncomment the following line to confirm the Database access information
#dbSecretValuesJson 

In [10]:
from sqlalchemy import create_engine
# Postgres username, password, and database name
POSTGRES_ADDRESS = dbSecretValuesJson['POSTGRES_ADDRESS'] ## INSERT YOUR DB ADDRESS IF IT'S NOT ON PANOPLY
POSTGRES_PORT = int(dbSecretValuesJson['POSTGRES_PORT'])
POSTGRES_USERNAME = dbSecretValuesJson['POSTGRES_USERNAME'] ## CHANGE THIS TO YOUR PANOPLY/POSTGRES USERNAME
POSTGRES_PASSWORD = dbSecretValuesJson['POSTGRES_PASSWORD'] ## CHANGE THIS TO YOUR PANOPLY/POSTGRES PASSWORD POSTGRES_DBNAME = 'database' ## CHANGE THIS TO YOUR DATABASE NAME
POSTGRES_DBNAME = dbSecretValuesJson['POSTGRES_DBNAME']
# A long string that contains the necessary Postgres login information
# postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(
#     username=POSTGRES_USERNAME,
#     password=POSTGRES_PASSWORD,
#     ipaddress=POSTGRES_ADDRESS,
#     port=POSTGRES_PORT,
#     dbname=POSTGRES_DBNAME))
postgres_str = f"postgresql://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_ADDRESS}/{POSTGRES_DBNAME}"

engine = create_engine(postgres_str)

In the follow code section, altitude_m is the surveyed height of building

In [11]:
sql_string = '''select * from building_detailed_info_within_ajjits_selected_tracts where bldgid = 59026'''
pd.read_sql_query(sql_string, engine)

,bldgid,egid,shape_area,altitude_m,rooftype,shape_length,border,geoid,name,namelsad,pct999,pct995,pct990,pct950
0,59026,DC00066551,3007.748522,59.5,Shed,512.80576,01060000000200000001030000000100000006000000BE...,11001005500,55,Census Tract 55,57.56059,57.5,55.61,52.71


From within lidarbuildingmatchtest database, las_grnd_coord, lidar_z, intensity, classification, over_lap, scan_angle, synthetic, withheld are the from the lidar data. 

##Explanation of columns:
* lidar_z: lidar measure Z axis value, or height per LIDAR measurement
* Intensity: return strength of the laster pulse https://desktop.arcgis.com/en/arcmap/10.3/manage-data/las-dataset/what-is-intensity-data-.htm
* Classification: 0 Never classified, 1 Unassigned, 2 Ground, 3 Low Vegetation, 4 Medium Vegation 5 High Vegatation, 5 High Veg, 6 Building, 7 Low Point etc. https://desktop.arcgis.com/en/arcmap/10.3/manage-data/las-dataset/lidar-point-classification.htm#ESRI_SECTION1_570719D89812478598FB633D71EBAD06 
* Scan angle: this is a categorical classication, with 0 being on target directly below instrument and larger number being off axis


In [12]:
sql_string = '''select las_grnd_coord, lidar_z, intensity, classification, scan_angle, synthetic, withheld from lidarbuildingmatchtest where bldgid = 59026'''
las_within_bldg_border_df = pd.read_sql_query(sql_string, engine)
las_within_bldg_border_df.describe()

,lidar_z,intensity,classification,scan_angle,synthetic,withheld
count,9942.000000,9942.000000,9942.000000,9942.000000,9942.0,9942.0
mean,43.609102,22272.492255,5.834440,6.090525,0.0,0.0
std,5.323083,12317.469069,0.795286,2.694791,0.0,0.0
min,28.840000,0.000000,1.000000,3.000000,0.0,0.0
25%,39.580000,11122.000000,6.000000,4.000000,0.0,0.0
50%,44.720000,24522.000000,6.000000,6.000000,0.0,0.0
75%,45.840000,28810.000000,6.000000,6.000000,0.0,0.0
max,57.840000,65535.000000,9.000000,11.000000,0.0,0.0


In [13]:
#Get the 99.5 percentile, 99 percentil, and 95 percentile of the lidar_z reading
las_qt_results_df = las_within_bldg_border_df[['lidar_z']].quantile([.999, .995, .99, .95, .90])
las_qt_results_df

,lidar_z
0.999,57.56059
0.995,57.50000
0.990,55.61000
0.950,52.71000
0.900,52.61000


In [14]:
las_qt_results_df.loc[.999:.900, 'lidar_z']

0.999    57.56059
0.995    57.50000
0.990    55.61000
0.950    52.71000
0.900    52.61000
Name: lidar_z, dtype: float64

In [15]:
pct999 = las_qt_results_df.loc[.999, 'lidar_z']
pct995 = las_qt_results_df.loc[.995, 'lidar_z']
pct990 = las_qt_results_df.loc[.990, 'lidar_z']
pct950 = las_qt_results_df.loc[.950, 'lidar_z']
pct900 = las_qt_results_df.loc[.900, 'lidar_z']
[pct999, pct995, pct990, pct950, pct900]

[57.56059000000001, 57.5, 55.61, 52.71, 52.61]

In [16]:
from sqlalchemy import update 
from geoalchemy2 import Geometry
from sqlalchemy.dialects import postgresql

In [17]:
connection = engine.connect()
metadata = db.MetaData()
building_detailed_info_table = db.Table('building_detailed_info_within_ajjits_selected_tracts', metadata, autoload=True, autoload_with=engine)

In [18]:
query = db.update(building_detailed_info_table).values({'pct999': pct999, 'pct995': pct995, 'pct990': pct990, 'pct950': pct950})
query = query.where(building_detailed_info_table.columns.bldgid == 59026)
print(query.compile(dialect=postgresql.dialect()))

UPDATE building_detailed_info_within_ajjits_selected_tracts SET pct999=%(pct999)s, pct995=%(pct995)s, pct990=%(pct990)s, pct950=%(pct950)s WHERE building_detailed_info_within_ajjits_selected_tracts.bldgid = %(bldgid_1)s


In [19]:
ResultProxy = connection.execute(query)
ResultProxy

In [20]:
# for row in ResultProxy:
#     print(row)

In [21]:
results = connection.execute(db.select([building_detailed_info_table]).where(building_detailed_info_table.columns.bldgid == 59026)).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df.head(4)

,bldgid,egid,shape_area,altitude_m,rooftype,shape_length,border,geoid,name,namelsad,pct999,pct995,pct990,pct950
0,59026,DC00066551,3007.748522,59.5,Shed,512.80576,01060000000200000001030000000100000006000000be...,11001005500,55,Census Tract 55,57.56059,57.5,55.61,52.71


In [22]:
#terminate connections and drop the db engine
engine.dispose()